In [227]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [228]:
# setup the logger
from pytz import timezone
import logging
import datetime
import sys
import os

logger = logging.getLogger(__name__)

def create_logger():
    if not os.path.exists('logs'):
        os.mkdir('logs')
        
    logging.Formatter.converter = lambda *args: datetime.datetime.now(tz=timezone('Europe/Bucharest')).timetuple()

    logfilename = datetime.datetime.now().strftime('logs/mylogfile_%d_%m_%Y.log')

    logger.handlers.clear()

    c_handler = logging.StreamHandler(sys.stdout)
    f_handler = logging.FileHandler(logfilename)
    c_handler.setLevel(logging.INFO)
    f_handler.setLevel(logging.INFO)

    c_format = logging.Formatter('%(asctime)s - [%(levelname)s] - %(message)s', datefmt='%d/%m/%Y %I:%M:%S %p')
    f_format = logging.Formatter('%(asctime)s - [%(levelname)s] - %(message)s', datefmt='%d/%m/%Y %I:%M:%S %p')
    c_handler.setFormatter(c_format)
    f_handler.setFormatter(f_format)

    logger.addHandler(c_handler)
    logger.addHandler(f_handler)

def log_msg(*args):
    msg = ""
    for arg in args:
        msg += str(arg)
    logger.info(msg)
    
def log_err(*args):
    msg = ""
    for arg in args:
        msg += str(arg)
    logger.error(msg)    

def log_start_session():
    logger.info('')
    logger.info('')
    logger.info('============================================================================================')
    logger.info('                                   Session Started')
    logger.info('============================================================================================')
    logger.info('')
    
def log_end_session():
    logger.info('')
    logger.info('')
    logger.info('============================================================================================')
    logger.info('                                   Session Ended')
    logger.info('============================================================================================')
    logger.info('')

# setup the callback method for showing updates after each epoch
def show_history(history):
    return pd.DataFrame(data=history)

create_logger()

log_start_session()

In [229]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, FunctionTransformer
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [230]:
log_msg("Import the dataset /kaggle/input/sii-competition/train.json")

df = pd.read_json("./input/train.json")

log_msg("Finished importing dataset")

In [231]:
df

,marca,model,an,km,putere,cutie_de_viteze,combustibil,capacitate_cilindrica,transmisie,caroserie,culoare,optiuni_culoare,addons,pret,id
0,Opel,Opel_Astra,2008,283100,115,Manuala,Benzina + CNG/GPL,1598,Fata,Masina de oras,Gri,Metalizata,"[Radio, Sistem audio, Control vocal, Climatron...",3650,0
1,Volkswagen,Volkswagen_Tiguan,2017,215994,190,Automata,Diesel,1968,4x4 (automat),SUV,Negru,Metalizata,"[Apple Carplay, Bluetooth, Radio, Sistem navig...",17950,1
2,Ford,Ford_Transit,2011,226000,116,Manuala,Diesel,2402,Fata,Monovolum,Alb,Fara,"[Radio, Aer conditionat, Cotiera (fata), Volan...",9500,2
3,Audi,Audi_A6,2013,290000,180,Automata,Benzina,1984,Fata,Sedan,Negru,Fara,"[Bluetooth, Radio, Sistem hands-free, Port USB...",11750,3
4,Land Rover,Land Rover_Discovery,2018,116628,258,Automata,Diesel,2993,4x4 (automat),SUV,Negru,Metalizata,"[Bluetooth, Radio, Sistem hands-free, Port USB...",38675,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20240,Volkswagen,Volkswagen_Polo,2018,52000,95,Automata,Benzina,999,Fata,Compacta,Gri,Mat,"[Radio, Sistem audio, Aer conditionat, Tapiter...",14756,20240
20241,Fiat,Fiat_Doblo,2017,124000,95,Manuala,Diesel,1248,Fata,Monovolum,Alb,Fara,"[Radio, Port USB, Cotiera (fata), Volan piele,...",10750,20241
20242,Skoda,Skoda_Octavia,2015,235170,150,Automata,Diesel,1968,Fata,Combi,Albastru,Fara,"[Bluetooth, Radio, Port USB, Wireless charging...",9485,20242
20243,Mercedes-Benz,Mercedes-Benz_GLE Coupe,2019,64390,390,Automata,Benzina,2996,4x4 (automat),SUV,Gri,Metalizata,"[Bluetooth, Radio, Sistem hands-free, Port USB...",62951,20243


In [232]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20245 entries, 0 to 20244
Data columns (total 15 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   marca                  20245 non-null  object
 1   model                  20245 non-null  object
 2   an                     20245 non-null  int64 
 3   km                     20245 non-null  int64 
 4   putere                 20245 non-null  int64 
 5   cutie_de_viteze        20245 non-null  object
 6   combustibil            20245 non-null  object
 7   capacitate_cilindrica  20245 non-null  int64 
 8   transmisie             20245 non-null  object
 9   caroserie              20245 non-null  object
 10  culoare                20245 non-null  object
 11  optiuni_culoare        20245 non-null  object
 12  addons                 20245 non-null  object
 13  pret                   20245 non-null  int64 
 14  id                     20245 non-null  int64 
dtypes: int64(6), object

In [233]:
!pip install seaborn


[notice] A new release of pip is available: 23.0 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [234]:
import matplotlib.pyplot as plt
import seaborn as sns

In [235]:
unique_addons = set(addon for addons_list in df['addons'] for addon in addons_list)

In [236]:
unique_addons

{'',
 'ABS',
 'Active lane control assistant',
 'Adaptive Cruise Control Predictive',
 'Aer conditionat',
 'Airbag central sofer si pasager',
 'Airbag cortina spate',
 'Airbag genunchi pasager',
 'Airbag genunchi sofer',
 'Airbag lateral sofer',
 'Airbag lateral sofer si pasager',
 'Airbag scaun pasager',
 'Airbag sofer',
 'Airbag-uri cap fata',
 'Airbag-uri cap spate',
 'Airbag-uri frontale pasageri spate',
 'Airbag-uri laterale spate',
 'Android Auto',
 'Anvelope Vara',
 'Anvelope iarna',
 'Aparatoare vant',
 'Apelare automata 112',
 'Apple Carplay',
 'Asistenta ambuteiaj',
 'Asistenta faza lunga',
 'Asistenta in panta',
 'Asistenta in rampa',
 'Asistenta la franare',
 'Asistenta la parcare',
 'Asistenta schimbare banda',
 'Asistenta telefonica urgenta',
 'Autocolant',
 'Avertizare sonora pre-coliziune',
 'Avertizare unghi mort',
 'Bluetooth',
 'Bullbar',
 'Cablu incarcare masina electrica',
 'Camera video 360º',
 'Camera video spate',
 'Cauciucuri Run-flat',
 'Cauciucuri all season'

In [237]:
len(unique_addons)

183

In [238]:
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.utils.validation import check_is_fitted# The ColumnsSelector class inherits from the sklearn.base classes 

In [239]:
class RemoveColumnsTransformer(BaseEstimator, TransformerMixin):
    def __init__(self, columns):
        # save the features list internally in the class
        self.columns = columns
        
    def fit(self, X, y = None):
        return self    
    
    def transform(self, X, y = None):
        # return the dataframe with the specified features
        return X[self.columns]

In [240]:
from sklearn.preprocessing import OneHotEncoder

class OHETransformer(BaseEstimator, TransformerMixin):
    def __init__(self, column):
        # save the features list internally in the class
        self.column = column
        self.one_hot_encoder = OneHotEncoder(sparse=False, handle_unknown='ignore')
        self.categories_ = None  # Store the categories during fit

        
    def fit(self, X, y = None):
        self.one_hot_encoder.fit(X[[self.column]])
        self.categories_ = self.one_hot_encoder.categories_
        return self
    
    def transform(self, X, y = None):
                # Transform the test data using the categories from the training data
        one_hot_encoded = pd.DataFrame(
            self.one_hot_encoder.transform(X[[self.column]]),
            columns=self.one_hot_encoder.get_feature_names_out([self.column])
        )

        one_hot_encoded.index = X.index
        
        # Concatenate the one-hot encoded columns with the original DataFrame
        X = pd.concat([X, one_hot_encoded], axis=1)

        X = X.drop(columns = [self.column])
        return X

In [241]:
class BinaryTransformer(BaseEstimator, TransformerMixin):
    def __init__(self, column, translations):
        # save the features list internally in the class
        self.column = column
        self.translations = translations
        
    def fit(self, X, y = None):
        return self    
    
    def transform(self, X, y = None):
        # return the dataframe with the specified features
        X[self.column] = X[self.column].map(self.translations)
        return X


In [242]:
class StringToIntTransformer(BaseEstimator, TransformerMixin):
    def __init__(self, column):
        # save the features list internally in the class
        self.column = column
        
    def fit(self, X, y = None):
        return self    
    
    def transform(self, X, y = None):
        # return the dataframe with the specified features
        X[self.column] = X[self.column].str.replace(r'm3', '', regex=True).fillna(0)
        X[self.column] = X[self.column].str.replace(r'[^0-9]', '', regex=True).fillna(0).astype(int)
        return X

In [243]:
class LowerCaseTransformer(BaseEstimator, TransformerMixin):
    def __init__(self, column = None):
        # save the features list internally in the class
        self.column = column
        
    def fit(self, X, y = None):
        return self    
    
    def transform(self, X, y = None):
        # return the dataframe with the specified features
        X[self.column] = X[self.column].str.replace('_', ' ')
        X[self.column] = X[self.column].str.lower()
        return X

In [244]:
from gensim.models import Word2Vec

class Word2VecNameTransformer(BaseEstimator, TransformerMixin):
    def __init__(self):
        self.vector_size = 50
        self.window = 3
        self.min_count = 1
        self.workers = 1
        self.word2vec_model = None
        
    def fit(self, X, y = None):
        car_names = X['model']
        tokenized_names = [name.split() for name in car_names]
        self.word2vec_model = Word2Vec(sentences=tokenized_names, vector_size=self.vector_size, window=self.window,
                                      min_count=self.min_count, workers=self.workers)
        return self
    
    def transform(self, X, y = None):
        car_names = X['model']
        tokenized_names = [name.split() for name in car_names]
        
        vectorized_names = [self.word2vec_model.wv[name] for name in tokenized_names]
        vectorized_names = [vectorized_names[i].mean(axis = 0) for i in range(0, len(vectorized_names))]
        
        vector_df = pd.DataFrame(vectorized_names, columns=[f"model_{i+1}" for i in range(self.word2vec_model.vector_size)])
        
        vector_df.index = X.index 
        
        X = pd.concat([X, vector_df], axis=1)
        X = X.drop(columns = ['model'])
        
        return X


In [245]:
class StandardScalerTransformer(BaseEstimator, TransformerMixin):
    def __init__(self, columns):
        self.scaler = StandardScaler()
        self.columns = columns
        
    def fit(self, X, y = None):
        self.scaler.fit(X[self.columns])
        return self
    
    def transform(self, X, y = None):
        X[self.columns] = self.scaler.transform(X[self.columns])
        return X

In [246]:
class AddonsTransformer(BaseEstimator, TransformerMixin):
    def __init__(self):
        return
    
    def fit(self, X, y = None):
        return self
    
    def transform(self, X, y = None):
        for addon in unique_addons:
            X[addon] = X['addons'].apply(lambda addons_list: 1 if addon in addons_list else 0)
        X.drop('addons', axis=1, inplace=True)
        return X

In [247]:
important_columns = [
    'model',
    'an',
    'km',
    'putere',
    'cutie_de_viteze',
    'combustibil',
    'capacitate_cilindrica',
    'transmisie',
    'caroserie',
    'culoare',
    'optiuni_culoare',
    'addons',
    'pret'
]

In [248]:
from sklearn.preprocessing import StandardScaler

pipe = Pipeline(
    steps = [
        ('select_columns', RemoveColumnsTransformer(important_columns)),
        ('1_hot_combustibil', OHETransformer('combustibil')),
        ('1_hot_transmisie', OHETransformer('transmisie')),
        ('1_hot_caroserie', OHETransformer('caroserie')),
        ('1_hot_culoare', OHETransformer('culoare')),
        ('1_hot_optiuni_culoare', OHETransformer('optiuni_culoare')),
        ('binary_cutie_de_viteze', BinaryTransformer('cutie_de_viteze', {'Automata': 1, 'Manuala': 0})),
        ('name_creation', LowerCaseTransformer('model')),
        ('name_embedding', Word2VecNameTransformer()),
        ('addon_encoder', AddonsTransformer()),
    ])

In [249]:
df_cleaned = pipe.fit_transform(df)

c:\Users\Dominic\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\preprocessing\_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
c:\Users\Dominic\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\preprocessing\_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
c:\Users\Dominic\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\preprocessing\_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
c:\Users\Dominic\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\pr

In [250]:
df_cleaned

,an,km,putere,cutie_de_viteze,capacitate_cilindrica,pret,combustibil_Benzina,combustibil_Benzina + CNG/GPL,combustibil_Diesel,combustibil_Hibrid,...,Incalzire cu reglare automata,Asistenta in rampa,Active lane control assistant,Sistem asistenta intersectie,Incalzire scaun sofer,Jante aliaj 21,Iluminare interioara LED,Top electric,Tapiterie alcantara,Lane assist
0,2008,283100,115,0,1598,3650,0.0,1.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
1,2017,215994,190,1,1968,17950,0.0,0.0,1.0,0.0,...,0,1,0,1,1,0,1,0,1,1
2,2011,226000,116,0,2402,9500,0.0,0.0,1.0,0.0,...,0,0,0,0,0,0,0,0,0,0
3,2013,290000,180,1,1984,11750,1.0,0.0,0.0,0.0,...,0,1,0,0,1,0,0,0,0,0
4,2018,116628,258,1,2993,38675,0.0,0.0,1.0,0.0,...,1,1,0,0,1,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20240,2018,52000,95,1,999,14756,1.0,0.0,0.0,0.0,...,0,1,0,0,0,0,0,0,0,0
20241,2017,124000,95,0,1248,10750,0.0,0.0,1.0,0.0,...,0,0,0,0,0,0,0,0,0,0
20242,2015,235170,150,1,1968,9485,0.0,0.0,1.0,0.0,...,0,1,0,0,1,0,0,0,0,1
20243,2019,64390,390,1,2996,62951,1.0,0.0,0.0,0.0,...,0,1,0,0,1,1,0,0,0,0


In [251]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import r2_score


In [252]:
X = df_cleaned.drop('pret', axis=1)
y = df_cleaned['pret']

In [253]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [254]:
# RandomForestRegressor Model
model = RandomForestRegressor(criterion='squared_error', max_depth=200, n_estimators=200)
model.fit(X_train, y_train)

# Predictions
y_pred = model.predict(X_test)

# R2 Score
r2 = r2_score(y_test, y_pred)
print(f'R2 Score: {r2}')

R2 Score: 0.9535989921354655


In [255]:
df_test = pd.read_json('./input/test.json')

In [256]:
df_test

,marca,model,an,km,putere,cutie_de_viteze,combustibil,capacitate_cilindrica,transmisie,caroserie,culoare,optiuni_culoare,addons,pret,id
0,BMW,BMW_Seria 7,2014,171000,313,Automata,Diesel,2993,4x4 (automat),Sedan,Gri,Metalizata,"[Bluetooth, Radio, Sistem hands-free, Port USB...",NaN,20245
1,Peugeot,Peugeot_508,2013,327000,163,Automata,Diesel,1997,4x4 (automat),Sedan,Albastru,Fara,"[Android Auto, Bluetooth, Radio, Sistem hands-...",NaN,20246
2,Ford,Ford_Focus,2013,219000,115,Manuala,Diesel,1560,Fata,Combi,Argint,Fara,"[Bluetooth, Radio, Sistem hands-free, Sistem n...",NaN,20247
3,Dacia,Dacia_Logan,2006,260000,75,Manuala,Benzina,1390,Fata,Sedan,Verde,Fara,"[Sistem navigatie, Senzori parcare spate, Ogli...",NaN,20248
4,Mercedes-Benz,Mercedes-Benz_Vito,2018,49767,163,Automata,Diesel,2143,4x4 (automat),Monovolum,Gri,Metalizata,"[Bluetooth, Radio, Sistem hands-free, Port USB...",NaN,20249
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5040,Opel,Opel_Zafira,2006,343000,120,Manuala,Diesel,1910,Fata,Monovolum,Negru,Metalizata,"[Radio, Sistem audio, Aer conditionat, Tapiter...",NaN,25285
5041,BMW,BMW_X3,2022,24000,190,Automata,Diesel,1995,4x4 (automat),SUV,Alte culori,Metalizata,"[Apple Carplay, Android Auto, Bluetooth, Radio...",NaN,25286
5042,Audi,Audi_A5,2008,138000,170,Manuala,Benzina,1798,Fata,Coupe,Negru,Metalizata,"[Bluetooth, Radio, Sistem audio, Climatronic 2...",NaN,25287
5043,Fiat,Fiat_Stilo,2001,243000,170,Automata,Benzina,2446,Fata,Coupe,Rosu,Mat,"[Radio, Sistem navigatie, Control vocal, Clima...",NaN,25288


In [257]:
df_test_copy = df_test.copy()

In [258]:
df_test_clean = pipe.transform(df_test_copy)

C:\Users\Dominic\AppData\Local\Temp\ipykernel_28548\3534813215.py:10: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  X[addon] = X['addons'].apply(lambda addons_list: 1 if addon in addons_list else 0)
C:\Users\Dominic\AppData\Local\Temp\ipykernel_28548\3534813215.py:10: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  X[addon] = X['addons'].apply(lambda addons_list: 1 if addon in addons_list else 0)
C:\Users\Dominic\AppData\Local\Temp\ipykernel_28548\3534813215.py:10: PerformanceWarning: DataFrame is highly fragmented.  This is usual

In [259]:
df_test_clean

,an,km,putere,cutie_de_viteze,capacitate_cilindrica,pret,combustibil_Benzina,combustibil_Benzina + CNG/GPL,combustibil_Diesel,combustibil_Hibrid,...,Incalzire cu reglare automata,Asistenta in rampa,Active lane control assistant,Sistem asistenta intersectie,Incalzire scaun sofer,Jante aliaj 21,Iluminare interioara LED,Top electric,Tapiterie alcantara,Lane assist
0,2014,171000,313,1,2993,NaN,0.0,0.0,1.0,0.0,...,1,1,0,1,1,0,1,0,0,1
1,2013,327000,163,1,1997,NaN,0.0,0.0,1.0,0.0,...,1,1,0,0,1,0,0,1,0,0
2,2013,219000,115,0,1560,NaN,0.0,0.0,1.0,0.0,...,0,0,0,0,0,0,0,0,0,0
3,2006,260000,75,0,1390,NaN,1.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
4,2018,49767,163,1,2143,NaN,0.0,0.0,1.0,0.0,...,0,1,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5040,2006,343000,120,0,1910,NaN,0.0,0.0,1.0,0.0,...,0,0,0,0,0,0,0,0,0,0
5041,2022,24000,190,1,1995,NaN,0.0,0.0,1.0,0.0,...,0,1,0,0,1,0,1,0,0,1
5042,2008,138000,170,0,1798,NaN,1.0,0.0,0.0,0.0,...,0,0,0,0,1,0,0,0,1,0
5043,2001,243000,170,1,2446,NaN,1.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0


In [260]:
X_test = df_test_clean.drop('pret', axis=1)

In [261]:
X_test

,an,km,putere,cutie_de_viteze,capacitate_cilindrica,combustibil_Benzina,combustibil_Benzina + CNG/GPL,combustibil_Diesel,combustibil_Hibrid,transmisie_4x4 (automat),...,Incalzire cu reglare automata,Asistenta in rampa,Active lane control assistant,Sistem asistenta intersectie,Incalzire scaun sofer,Jante aliaj 21,Iluminare interioara LED,Top electric,Tapiterie alcantara,Lane assist
0,2014,171000,313,1,2993,0.0,0.0,1.0,0.0,1.0,...,1,1,0,1,1,0,1,0,0,1
1,2013,327000,163,1,1997,0.0,0.0,1.0,0.0,1.0,...,1,1,0,0,1,0,0,1,0,0
2,2013,219000,115,0,1560,0.0,0.0,1.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
3,2006,260000,75,0,1390,1.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
4,2018,49767,163,1,2143,0.0,0.0,1.0,0.0,1.0,...,0,1,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5040,2006,343000,120,0,1910,0.0,0.0,1.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
5041,2022,24000,190,1,1995,0.0,0.0,1.0,0.0,1.0,...,0,1,0,0,1,0,1,0,0,1
5042,2008,138000,170,0,1798,1.0,0.0,0.0,0.0,0.0,...,0,0,0,0,1,0,0,0,1,0
5043,2001,243000,170,1,2446,1.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0


In [262]:
y_pred = model.predict(X_test)

In [263]:
y_pred

array([21012.985,  9862.18 ,  5556.935, ...,  7179.655,  4535.885,
       17018.315])

In [264]:
df_test['pret'] = y_pred

In [265]:
df_test

,marca,model,an,km,putere,cutie_de_viteze,combustibil,capacitate_cilindrica,transmisie,caroserie,culoare,optiuni_culoare,addons,pret,id
0,BMW,BMW_Seria 7,2014,171000,313,Automata,Diesel,2993,4x4 (automat),Sedan,Gri,Metalizata,"[Bluetooth, Radio, Sistem hands-free, Port USB...",21012.985,20245
1,Peugeot,Peugeot_508,2013,327000,163,Automata,Diesel,1997,4x4 (automat),Sedan,Albastru,Fara,"[Android Auto, Bluetooth, Radio, Sistem hands-...",9862.180,20246
2,Ford,Ford_Focus,2013,219000,115,Manuala,Diesel,1560,Fata,Combi,Argint,Fara,"[Bluetooth, Radio, Sistem hands-free, Sistem n...",5556.935,20247
3,Dacia,Dacia_Logan,2006,260000,75,Manuala,Benzina,1390,Fata,Sedan,Verde,Fara,"[Sistem navigatie, Senzori parcare spate, Ogli...",2243.620,20248
4,Mercedes-Benz,Mercedes-Benz_Vito,2018,49767,163,Automata,Diesel,2143,4x4 (automat),Monovolum,Gri,Metalizata,"[Bluetooth, Radio, Sistem hands-free, Port USB...",37938.595,20249
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5040,Opel,Opel_Zafira,2006,343000,120,Manuala,Diesel,1910,Fata,Monovolum,Negru,Metalizata,"[Radio, Sistem audio, Aer conditionat, Tapiter...",2896.675,25285
5041,BMW,BMW_X3,2022,24000,190,Automata,Diesel,1995,4x4 (automat),SUV,Alte culori,Metalizata,"[Apple Carplay, Android Auto, Bluetooth, Radio...",52783.380,25286
5042,Audi,Audi_A5,2008,138000,170,Manuala,Benzina,1798,Fata,Coupe,Negru,Metalizata,"[Bluetooth, Radio, Sistem audio, Climatronic 2...",7179.655,25287
5043,Fiat,Fiat_Stilo,2001,243000,170,Automata,Benzina,2446,Fata,Coupe,Rosu,Mat,"[Radio, Sistem navigatie, Control vocal, Clima...",4535.885,25288


In [266]:
df_test.to_json('./output/test.json', orient='records')